In [ ]:

import warnings,os
# remove warnings from ax
os.environ["PYTHONWARNINGS"] = "ignore"
warnings.filterwarnings(action='ignore', category=FutureWarning)
import ax
from optimpv import *
from optimpv.axBOtorch.axUtils import *
import pandas as pd
import numpy as np
from scipy import interpolate
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch, copy, os
from itertools import combinations
from ax.plot.contour import plot_contour
from ax.plot.trace import optimization_trace_single_method
from ax.service.ax_client import AxClient
from ax.utils.notebook.plotting import init_notebook_plotting, render
from botorch.models import SaasFullyBayesianSingleTaskGP, SingleTaskGP
# import logging

# from ray import train, tune
# from ray.tune.schedulers import AsyncHyperBandScheduler
# from ray.tune.search.ax import AxSearch



init_notebook_plotting()

# Suppress FutureWarning messages
# warnings.simplefilter(action='ignore', category=FutureWarning)
# ##part of the message is also okay
warnings.filterwarnings('ignore') 
params = []

mun = FitParam(name = 'l2.mu_n', value = 8e-4, bounds = [1e-5,1e-2], values = None, start_value = None, log_scale = True, value_type = 'float', fscale = None, rescale = False, stepsize = None, display_name=r'$\mu_n$', unit='m$^2$ V$^{-1}$s$^{-1}$', axis_type = 'log', std = 0,encoding = None,force_log = False)
params.append(mun)

mup = FitParam(name = 'l2.mu_p', value = 5e-5, bounds = [1e-5,1e-2], values = None, start_value = None, log_scale = True, value_type = 'float', fscale = None, rescale = False, stepsize = None, display_name=r'$\mu_p$', unit=r'm$^2$ V$^{-1}$s$^{-1}$', axis_type = 'log', std = 0,encoding = None,force_log = False)
params.append(mup)

bulk_tr = FitParam(name = 'l2.N_t_bulk', value = 1e20, bounds = [1e19,1e21], values = None, start_value = None, log_scale = True, value_type = 'float', fscale = None, rescale = False, stepsize = None, display_name=r'$N_{T}$', unit=r'm$^{-3}$', axis_type = 'log', std = 0,encoding = None,force_log = False)
params.append(bulk_tr)

int_trap = FitParam(name = 'l1.N_t_int', value = 4e12, bounds = [1e11,1e13], values = None, start_value = None, log_scale = True, value_type = 'float', fscale = None, rescale = False, stepsize = None, display_name=r'$N_{T,int}^{ETL}$', unit='m$^{-2}$', axis_type = 'log', std = 0,encoding = None,force_log = False)
params.append(int_trap)

Nions = FitParam(name = 'l2.N_ions', value = 1e22, bounds = [1e20,5e22], type='range', values = None, start_value = None, log_scale = True, value_type = 'float', fscale = None, rescale = False, stepsize = None, display_name=r'$C_{ions}$', unit='m$^{-3}$', axis_type = 'log', std = 0,encoding = None,force_log = False)
params.append(Nions)

R_series = FitParam(name = 'R_series', value = 1e-4, bounds = [1e-5,1e-3], type='range', values = None, start_value = None, log_scale = True, value_type = 'float', fscale = None, rescale = False, stepsize = None, display_name=r'$R_{series}$', unit=r'$\Omega$ m$^2$', axis_type = 'log', std = 0,encoding = None,force_log = False)
params.append(R_series)

# original values
params_orig = copy.deepcopy(params)



In [ ]:
import pySIMsalabim as sim
from pySIMsalabim.experiments.JV_steady_state import *

session_path = os.path.join('/home/lecorre/Desktop/pySIMsalabim/', 'SIMsalabim','SimSS')
simss_device_parameters = os.path.join(session_path, 'simulation_setup.txt')

# Set the JV parameters
Gfracs = [0.1,0.5,1.0] # Fractions of the generation rate to simulate
# Gfracs = None
UUID = str(uuid.uuid4())

cmd_pars = []
for param in params:
    if param.name != 'l2.C_np_bulk' and param.name != 'offset_l2_l1.E_c' and param.name != 'offset_l2_l3.E_v' and param.name != 'Egap_l1.E_v' and param.name != 'offset_W_L.E_c' and param.name != 'l2.N_ions':
        cmd_pars.append({'par':param.name, 'val':str(param.value)})
    elif param.name == 'offset_l2_l1.E_c':
        cmd_pars.append({'par':'l1.E_c', 'val':str(3.9-param.value)})
        vv = 3.9-param.value
    elif param.name == 'l2.N_ions':
        cmd_pars.append({'par':'l2.N_cation', 'val':str(param.value)})
        cmd_pars.append({'par':'l2.N_anion', 'val':str(param.value)})
    elif param.name == 'l2.C_np_bulk':
        cmd_pars.append({'par':'l2.C_n_bulk', 'val':str(param.value)})
        cmd_pars.append({'par':'l2.C_p_bulk', 'val':str(param.value)})

    elif param.name == 'offset_l2_l3.E_v':
        cmd_pars.append({'par':'l3.E_v', 'val':str(5.53-param.value)})
    
    elif param.name == 'Egap_l1.E_v':
        cmd_pars.append({'par':'l1.E_v', 'val': str(vv+param.value)})
    
    elif param.name == 'offset_W_L.E_c':
        cmd_pars.append({'par':'W_L', 'val':str(vv-param.value)})


# Run the JV simulation
ret, mess = run_SS_JV(simss_device_parameters, session_path, JV_file_name = 'JV.dat', varFile= 'Var.dat',G_fracs = Gfracs, parallel = True, max_jobs = 3, UUID=UUID, cmd_pars=cmd_pars)

# import random noise
from numpy.random import default_rng
# save data for fitting
X,y = [],[]
if Gfracs is None:
    data = pd.read_csv(os.path.join(session_path, 'JV_'+UUID+'.dat'), sep=r'\s+') # Load the data
    Vext = np.asarray(data['Vext'].values)
    Jext = np.asarray(data['Jext'].values)
    G = np.ones_like(Vext)
    rng = default_rng()#
    noise = rng.standard_normal(Jext.shape) * 0.01 * Jext
    Jext = Jext + noise
    X= Vext
    y = Jext

    plt.figure()
    plt.plot(X,y)
    plt.show()
else:
    for Gfrac in Gfracs:
        data = pd.read_csv(os.path.join(session_path, 'JV_Gfrac_'+str(Gfrac)+'_'+UUID+'.dat'), sep=r'\s+') # Load the data
        Vext = np.asarray(data['Vext'].values)
        Jext = np.asarray(data['Jext'].values)
        G = np.ones_like(Vext)*Gfrac
        rng = default_rng()#
        noise = rng.standard_normal(Jext.shape) * 0.005 * Jext
        Jext = Jext + noise

        if len(X) == 0:
            X = np.vstack((Vext,G)).T
            y = Jext
        else:
            X = np.vstack((X,np.vstack((Vext,G)).T))
            y = np.hstack((y,Jext))

    # remove all the current where Jext is positive i.e. above Voc
    X = X[y<50]
    y = y[y<50]

    plt.figure()
    for Gfrac in Gfracs:
        plt.plot(X[X[:,1]==Gfrac,0],y[X[:,1]==Gfrac],label='Gfrac = '+str(Gfrac))
    plt.legend()
    plt.show()




In [3]:
from optimpv.DDfits.JVAgent import JVAgent
metric = 'mse'
# metric = 'mse'
# loss = 'log10'
# loss = 'linear'
loss = 'soft_l1'

jv = JVAgent(params, X, y, session_path, simss_device_parameters,parallel = True, max_jobs = 3, metric = metric, loss = loss)

In [4]:
from optimpv.axBOtorch.axBOtorchOptimizer import axBOtorchOptimizer
from botorch.acquisition.logei import (
    qLogExpectedImprovement,
    qLogNoisyExpectedImprovement,
)
from  botorch.acquisition.monte_carlo import qUpperConfidenceBound
from ax.modelbridge.transforms.standardize_y import StandardizeY
from ax.modelbridge.transforms.unit_x import UnitX
from ax.modelbridge.transforms.remove_fixed import RemoveFixed
from ax.modelbridge.transforms.log import Log
from ax.models.torch.botorch_modular.surrogate import Surrogate
#import single task GP
from botorch.models.gp_regression import SingleTaskGP
# SASS = SaasFullyBayesianSingleTaskGP
from botorch.models.fully_bayesian import SaasFullyBayesianSingleTaskGP

# surr = Surrogate(SingleTaskGP)
# model_kwargs_list = [{}]
# for i in range(3):
#     model_kwargs_list.append({'torch_device': torch.device("cuda" if torch.cuda.is_available() else "cpu"),'torch_dtype': torch.double,'botorch_acqf_class':qLogNoisyExpectedImprovement,'transforms':[RemoveFixed, Log,UnitX, StandardizeY],'acquisition_options':{'tau_relu':1e-7}})
# model_kwargs_list = [{},{'torch_device': torch.device("cuda" if torch.cuda.is_available() else "cpu"),'torch_dtype': torch.double,'botorch_acqf_class':qLogNoisyExpectedImprovement,'transforms':[RemoveFixed, Log,UnitX, StandardizeY],},{'torch_device': torch.device("cuda" if torch.cuda.is_available() else "cpu"),'torch_dtype': torch.double,'botorch_acqf_class':qUpperConfidenceBound,'transforms':[RemoveFixed, Log,UnitX, StandardizeY]},{'torch_device': torch.device("cuda" if torch.cuda.is_available() else "cpu"),'torch_dtype': torch.double,'botorch_acqf_class':qLogNoisyExpectedImprovement,'transforms':[RemoveFixed, Log,UnitX, StandardizeY]}]
# model_gen_kwargs_list = [None,{'n':2,'joint_optimization':True},{'n':2,'joint_optimization':True},{'n':2,'joint_optimization':True}]
model_gen_kwargs_list = None
# parameter_constraints = [f'l2.mu_p - l2.mu_n <= {0}']
parameter_constraints = None
# print(len(model_kwargs_list))
from gpytorch.kernels import MaternKernel
from gpytorch.kernels import RBFKernel
from gpytorch.constraints import Interval
#  acquisition_options
# prior = {'eta':100}
# acf_options
# optimizer = axBOtorchOptimizer(params = params, agents = jv, models = ['SOBOL','BOTORCH_MODULAR','BOTORCH_MODULAR','BOTORCH_MODULAR'],n_batches = [1,30,20,30], batch_size = [10,2,2,2], metrics = metric, minimize_list = True, thresholds = None, ax_client = None,  max_parallelism = 10,model_kwargs_list = model_kwargs_list, model_gen_kwargs_list = model_gen_kwargs_list, name = 'ax_opti',parameter_constraints = parameter_constraints)
# optimizer = axBOtorchOptimizer(params = params, agents = jv, models = ['SOBOL','BOTORCH_MODULAR','BOTORCH_MODULAR','BOTORCH_MODULAR'],n_batches = [1,30,20,30], batch_size = [100,2,2,2], metrics = metric, minimize_list = True, thresholds = None, ax_client = None,  max_parallelism = 10,model_kwargs_list = model_kwargs_list, model_gen_kwargs_list = model_gen_kwargs_list, name = 'ax_opti',parameter_constraints = parameter_constraints)
# from ax.models.torch.botorch_modular.utils import ModelConfig
# model_config = ModelConfig(covar_module_class='MaternKernel')
# from gpytorch.constraints import GreaterThan
# # (raw_lengthscale_constraint): Positive()
from ax.models.torch.botorch_modular.utils import ModelConfig
from ax.models.torch.botorch_modular.surrogate import SurrogateSpec
# from 
# model_config = ModelConfig(covar_module_class=MaternKernel, covar_module_options={'lengthscale_constraint':Interval(1e-6,100)})
# model_kwargs_list = [{},{'torch_device': torch.device("cuda" if torch.cuda.is_available() else "cpu"),'torch_dtype': torch.double,'botorch_acqf_class':qLogNoisyExpectedImprovement,'transforms':[RemoveFixed, Log,UnitX, StandardizeY],'surrogate':Surrogate(SingleTaskGP),'model_configs':model_config}]#,covar_module_optionsMaternKernellengthscale_constraint':Interval(1e-6,100)})}]
# model_configs = [None,ModelConfig(covar_module_class=MaternKernel, covar_module_options={'lengthscale_constraint':Interval(1e-6,100)},torch_device = torch.device("cuda" if torch.cuda.is_available() else "cpu"),torch_dtype = torch.double)]
# model_kwargs_list = {}
model_configs = [{},{}]
model_kwargs_list = [{},{'torch_device': torch.device("cuda" if torch.cuda.is_available() else "cpu"),'torch_dtype': torch.double,'botorch_acqf_class':qLogNoisyExpectedImprovement,'transforms':[RemoveFixed, Log,UnitX, StandardizeY],'surrogate_spec':SurrogateSpec(model_configs=[ModelConfig(botorch_model_class=SingleTaskGP,covar_module_class=MaternKernel,covar_module_options={'lengthscale_constraint':Interval(1e-6,100)})])}]
# n_batches = [1,140], batch_size = [40,4]                     
optimizer = axBOtorchOptimizer(params = params, agents = jv, models = ['SOBOL','BOTORCH_MODULAR'],n_batches = [1,20], batch_size = [40,4], ax_client = None,  max_parallelism = 100,
                    model_configs = model_configs, model_kwargs_list = model_kwargs_list, model_gen_kwargs_list = model_gen_kwargs_list, name = 'ax_opti',parameter_constraints = parameter_constraints,scheduler_logging_level = 0)


In [ ]:
optimizer.optimize(True)

In [ ]:
ax_client = optimizer.ax_client
best_parameters = ax_client.get_best_parameters()[0]
print(best_parameters)
jv.params_w(best_parameters,jv.params)
print(jv.get_SIMsalabim_clean_cmd(jv.params))
jv.package_SIMsalabim_files(jv.params,'simss')

In [ ]:
# plot the evolution of the optimization
render(ax_client.get_contour_plot(param_x="l2.mu_n", param_y="l2.mu_p", metric_name=optimizer.all_metrics[0]))
# render(ax_client.get_contour_plot(param_x="l2.N_t_bulk", param_y="l2.C_np_bulk", metric_name=metric))
from ax.plot.slice import plot_slice
model = ax_client.generation_strategy.model

# render(plot_slice(model=model, param_name="l2.mu_n", metric_name=optimizer.all_metrics[0]))
# render(plot_slice(model=model, param_name="l2.mu_p", metric_name=optimizer.all_metrics[0]))
render(plot_slice(model=model, param_name="l2.N_t_bulk", metric_name=optimizer.all_metrics[0]))
# render(plot_slice(model=model, param_name="l2.N_ions", metric_name=optimizer.all_metrics[0]))



In [ ]:
data = ax_client.experiment.fetch_data()

plt.plot(np.minimum.accumulate(data.df["mean"]), label="Best value seen so far")

# plt.yscale("log")


In [9]:
# # create dic with keys same as ax_client.experiment.trials[0].arm.parameters
# from ax.core.base_trial import TrialStatus as T
# dumdic = {}
# for key in ax_client.experiment.trials[0].arm.parameters.keys():
#     dumdic[key] = []

# # fill the dic with the values of the parameters
# for i in range(len(ax_client.experiment.trials)):
#     if ax_client.experiment.trials[i].status == T.COMPLETED:
#         for key in ax_client.experiment.trials[i].arm.parameters.keys():
#             dumdic[key].append(ax_client.experiment.trials[i].arm.parameters[key])


# data = ax_client.experiment.fetch_data().df

# target1 = data[data['metric_name'] == optimizer.all_metrics[0]]['mean']

# dumdic[optimizer.all_metrics[0]] = list(target1)


# dumdic['iteration'] = list(data[data['metric_name'] == optimizer.all_metrics[0]]['trial_index'])

# df = pd.DataFrame(dumdic)


# for par in params:
#     if par.name in df.columns:
#         if par.rescale:
#             if par.value_type == 'int':
#                 df[par.name] = df[par.name] * par.stepsize
#             else:
#                 df[par.name] = df[par.name] * par.fscale


In [ ]:
# get name of all parameters that are not 'fixed'
from optimpv.posterior.posterior import *
params_orig_dict = {}
for p in params_orig:
    params_orig_dict[p.name] = p.value
fig_dens, ax_dens = plot_density_exploration(params, optimizer.all_metrics[0], optimizer = optimizer, best_parameters = best_parameters, params_orig = params_orig_dict, optimizer_type = 'ax')


In [ ]:
# rerun the simulation with the best parameters
yfit = jv.run(parameters=ax_client.get_best_parameters()[0])
# print(jv.run_Ax(parameters=results.get_best_result(metric=metric,mode='min',filter_nan_and_inf=True).config))
plt.figure(figsize=(20,20))
for Gfrac in Gfracs:
    plt.plot(X[X[:,1]==Gfrac,0],y[X[:,1]==Gfrac],label='Gfrac = '+str(Gfrac))
    plt.plot(X[X[:,1]==Gfrac,0],yfit[X[:,1]==Gfrac],label='Gfrac = '+str(Gfrac)+' fit',linestyle='--')
plt.legend()
plt.show()

In [ ]:
from ax.modelbridge.cross_validation import cross_validate
from ax.plot.contour import interact_contour
from ax.plot.diagnostic import interact_cross_validation
cv_results = cross_validate(model)
render(interact_cross_validation(cv_results))

In [ ]:
# flush cuda memory
torch.cuda.empty_cache()

Nres = 10
objective_name = optimizer.all_metrics[0]
model = optimizer.ax_client.generation_strategy.model
# set 
    
fig, ax = devils_plot(params, Nres, objective_name, model, loss, best_parameters = best_parameters, params_orig = params_orig_dict, optimizer_type = 'ax')

In [ ]:
fig, ax = plot_1d_posteriors(params, Nres, objective_name, model, loss, best_parameters = best_parameters, params_orig = params_orig_dict, optimizer_type = 'ax')

In [ ]:
fig, ax = plot_1D_2D_posterior(params, 'l2.mu_n', 'l2.mu_p', 10, objective_name, model, loss, best_parameters = best_parameters, params_orig = params_orig_dict, optimizer_type = 'ax')

In [ ]:
from optimpv.general.general import inv_loss_function
dumdic = {}
# create a dic with the keys of the parameters
if isinstance(ax_client.experiment.trials[0], BatchTrial):# check if trial is a BatchTrial
    for key in ax_client.experiment.trials[0].arms[0].parameters.keys():
        dumdic[key] = []
    
    # fill the dic with the values of the parameters
    for i in range(len(ax_client.experiment.trials)):
        if ax_client.experiment.trials[i].status == T.COMPLETED:
            for arm in ax_client.experiment.trials[i].arms:
                for key in arm.parameters.keys():
                    dumdic[key].append(arm.parameters[key])
            # for key in ax_client.experiment.trials[i].arms[0].parameters.keys():
            #     dumdic[key].append(ax_client.experiment.trials[i].arms[0].parameters[key])
else:
    for key in ax_client.experiment.trials[0].arm.parameters.keys():
        dumdic[key] = []

    # fill the dic with the values of the parameters
    for i in range(len(ax_client.experiment.trials)):
        if ax_client.experiment.trials[i].status == T.COMPLETED:
            for key in ax_client.experiment.trials[i].arm.parameters.keys():
                dumdic[key].append(ax_client.experiment.trials[i].arm.parameters[key])


data = ax_client.experiment.fetch_data().df

target1 = data[data['metric_name'] == objective_name]['mean']

dumdic[objective_name] = list(inv_loss_function(target1,loss))


dumdic['iteration'] = list(data[data['metric_name'] == objective_name]['trial_index'])

df = pd.DataFrame(dumdic)
# make a second df with the 10 best results
df_best = df.nsmallest(10, objective_name)
# all objectivevalues below 3
# df_best = df[df[objective_name] < 3]
print(df_best)
import matplotlib.pyplot as plt
import matplotlib
plt.figure(figsize=(10, 6))

lognorm = matplotlib.colors.LogNorm(vmin=df[objective_name].min(), vmax=df[objective_name].max())

# params_name = ['l2.N_t_bulk', 'l2.mu_p']
# params_name = ['l2.N_t_bulk', 'l1.N_t_int']
# params_name = ['l2.N_t_bulk', 'l2.N_ions']
params_name = ['l2.mu_n', 'l2.mu_p']
sc = plt.scatter(df[params_name[0]], df[params_name[1]], c=df[objective_name], norm=lognorm, cmap='viridis_r')
plt.scatter(df_best[params_name[0]], df_best[params_name[1]], c='grey', marker='o', label='10 best results')
# plot best result
plt.scatter(best_parameters[params_name[0]], best_parameters[params_name[1]], c='blue', marker='x', label='Best result')
# plot initial points
plt.scatter(params_orig_dict[params_name[0]], params_orig_dict[params_name[1]], c='red', marker='x', label='Initial values')
plt.yscale('log')
plt.xscale('log')
plt.colorbar(sc, label='JV_JV_mse')
plt.xlabel(params_name[0])
plt.ylabel(params_name[1])
plt.grid(True, which='both', linestyle='--', linewidth=0.5)

# Make and density plot of df_best 1d

for i in range(len(params)):
    plt.figure()
    
    # hist with log scale y bins
    logbins = np.geomspace(params[i].bounds[0], params[i].bounds[1], 100)
    plt.hist(df_best[params[i].name], bins=logbins, alpha=0.5, label='10 best results', color='b')
    # kde plt
    sns.kdeplot(df_best[params[i].name], log_scale=True, label='10 best results', color='b')

    plt.vlines(best_parameters[params[i].name], 0, 5, colors='r', linestyles='dashed', label='Best result')
    plt.vlines(params_orig_dict[params[i].name], 0, 5, colors='k', linestyles='dashed', label='Initial values')
    # add median mean std and IQR to plot as vlines and shaded area
    plt.vlines(df_best[params[i].name].median(), 0, 5, colors='g', linestyles='dashed', label='Median')
    plt.vlines(df_best[params[i].name].mean(), 0, 5, colors='orange', linestyles='dashed', label='Mean')
    # plt.vlines(df_best[params[i].name].mean() + df_best[params[i].name].std(), 0, 3, colors='purple', linestyles='dashed', label='Mean + std')
    # plt.vlines(df_best[params[i].name].mean() - df_best[params[i].name].std(), 0, 3, colors='purple', linestyles='dashed', label='Mean - std')
    plt.fill_betweenx([0, 5], df_best[params[i].name].mean() - df_best[params[i].name].std(), df_best[params[i].name].mean() + df_best[params[i].name].std(), color='purple', alpha=0.3, label='Mean +/- std')
    # plt.fill_betweenx([0, 5], df_best[params[i].name].quantile(0.25), df_best[params[i].name].quantile(0.75), color='g', alpha=0.3, label='IQR')

    plt.legend()
    plt.xscale('log')
    plt.xlim(params[i].bounds[0], params[i].bounds[1])
plt.show()


In [17]:
# Clean up the output files (comment out if you want to keep the output files)
sim.clean_all_output(session_path)
sim.delete_folders('tmp',session_path)